In [1]:
import scipy.io

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

from kpnet.handler import *
from kpnet.network import *
from kpnet.signal import *
from kpnet.callback import *

In [2]:
time_interval = 2 ** 16
chunk = 2 ** 13

In [3]:
filelist  = pd.read_csv('data/hfd5/desc/data_gen.csv')

In [4]:
filelist

,Filename,N,alpha,beta,gamma
0,cluster1.hfd5,64,0.050,0.3000,2.2
1,cluster2.hfd5,64,0.050,0.2500,2.2
2,cluster3.hfd5,64,0.125,0.0010,2.2
3,cluster4.hfd5,64,0.125,0.0012,2.2
4,cluster5.hfd5,64,0.060,0.0900,2.2
5,trans1.hfd5,64,0.200,0.4000,2.2
6,trans2.hfd5,64,0.125,0.0015,2.2
7,trans3.hfd5,64,0.100,0.0015,2.2
8,period1.hfd5,64,0.400,0.4000,2.2
9,period2.hfd5,64,0.200,0.0280,2.2


In [ ]:
np.random.seed(0)
N = 64
array = np.zeros(shape=(time_interval, N, 1))
signal = SwitchSignal([2000, time_interval], [RandomUnitSignal(N), ZeroSignal(N)])
for i in range(time_interval):
    array[i] = signal[i]
d = {}
d['signal'] = array
scipy.io.savemat('data/mat/signal.mat', d)

In [ ]:
for i in tqdm(range(filelist.shape[0])):
    np.random.seed(0)
    N = filelist['N'][i]
    net = KPNetworkTanh(N, filelist['alpha'][i], filelist['beta'][i], filelist['gamma'][i])
    callbacks = {}
    callbacks['W'] = WeightsCallback(time_interval, N, chunk, 'data/hfd5/' + filelist['Filename'][i])
    signal = SwitchSignal([2000, time_interval], [RandomUnitSignal(N), ZeroSignal(N)])
    handler = ComputationHandler(time_interval)
    results = handler.run(net, signal, callbacks)